# Non-linear Panel Models
---
## Note:
- This notebook uses the "Crime from North Carolina" dataset from notebook 2
- `statsmodels`'s conditional logit implementation fails to converge with the Panel101 "toy dataset"
- The "toy dataset" does not have sufficient variation within groups. It also suffers from high multicollinearity between variables within groups (see Section 1.a below)

## Overview (and pre-analysis plan)
In this notebook, we will:
- Fit multiple panel models to non-linear data (i.e. data with binary outcomes)
- Models include:
    - Pooled logit
    - Conditional logit (country fixed effects)
    - Conditional logit (year fixed effects)
    - LPM
    - LPM (country fixed effects)
    - LPM (year fixed effects)
    - LPM (two-way fixed effects)

In [1]:
import pandas as pd

import statsmodels.api as sm

from linearmodels import PanelOLS

from statsmodels.discrete.conditional_models import ConditionalLogit
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1.a Data prep

In [2]:
# Load data (from http://www.princeton.edu/~otorres/LogitR101.pdf )

data = pd.read_stata('http://dss.princeton.edu/training/Panel101.dta')
data

,country,year,y,y_bin,x1,x2,x3,opinion,op
0,A,1990,1.342788e+09,1.0,0.277904,-1.107956,0.282554,Str agree,1.0
1,A,1991,-1.899661e+09,0.0,0.320685,-0.948720,0.492538,Disag,0.0
2,A,1992,-1.123436e+07,0.0,0.363466,-0.789484,0.702523,Disag,0.0
3,A,1993,2.645775e+09,1.0,0.246144,-0.885533,-0.094391,Disag,0.0
4,A,1994,3.008335e+09,1.0,0.424623,-0.729768,0.946131,Disag,0.0
...,...,...,...,...,...,...,...,...,...
65,G,1995,1.323696e+09,1.0,1.087186,-1.409817,2.829808,Str disag,0.0
66,G,1996,2.545242e+08,1.0,0.781075,-1.328000,4.278224,Str agree,1.0
67,G,1997,3.297033e+09,1.0,1.257879,-1.577367,4.587326,Disag,0.0
68,G,1998,3.011821e+09,1.0,1.242777,-1.601218,6.113762,Disag,0.0


In [3]:
# Seperate exogenous columns, endogenous columns, 
# country column, and year column

exog = data[['x1', 'x2', 'x3']]
endog = data['y_bin'].astype(int)
countries = data['country']
years = data['year']

In [4]:
# The "toy dataset" has very few observations per group

data.groupby(by='country').size()

country
A    10
B    10
C    10
D    10
E    10
F    10
G    10
dtype: int64

In [5]:
# Get variance within each country for y_bin, x1, x2, and x3

within_country_var = data[['y_bin', 'x1', 'x2', 'x3', 'country']].groupby(by='country').var()
within_country_var

,y_bin,x1,x2,x3
country,,,,
A,0.177778,0.016893,0.019946,0.345907
B,0.277778,0.111110,0.014658,0.157842
C,0.233333,0.013766,0.004341,0.368436
D,0.100000,0.059227,0.030829,0.443879
E,0.100000,0.166607,0.114101,0.656200
F,0.100000,0.304088,0.029110,0.397462
G,0.100000,0.037319,0.012824,4.367728


In [6]:
# Check multicollinearity within groups using variance inflation factor

def get_vif(data):
    """Returns vif of each column in `data` as a np.array.
    """
    vif = []
    exog = data.iloc[:,:-1].to_numpy()
    for i in range(len(data.columns)-1):
        vif.append(variance_inflation_factor(exog, i))
    vif = (pd.DataFrame(vif)
             .set_index(pd.Index(['x1', 'x2', 'x3'],
                                 name='exog_var')))
    return vif


vif_country_var = (
    data[['x1', 'x2', 'x3', 'country']]
        .groupby(by='country')
        .apply(lambda x: get_vif(x))
        .pipe(pd.DataFrame)
        .rename(columns={0:'vif'})
        .explode(column='vif')
        .reset_index(level=1)
        .pivot(columns='exog_var')
)
vif_country_var

vif                      
exog_var          x1          x2        x3
country                                   
A           8.139231    6.271669  3.250608
B           4.346689    4.174591  1.133515
C         174.555936  181.030677  1.818913
D           2.367951    2.604404  1.542439
E           9.594652    8.652426  2.383203
F           2.481675    2.838929  1.238919
G          51.182267   53.419598  4.493741

### Note:
- As we see above, countries C, G, E, and A and have high multicollinearity between variables x1 and x2
- I've tried multiple hyperaparameters (intial values, numerical methods) for `statsmodels`'s conditional logit implementation
- `statsmodels`'s conditional logit fails to converge across all my attempts
- This example demonstrates that numerical issues **can differ across implementations** (e.g. R's clogit has no issues with this "toy dataset")
- Let's use a real dataset instead: "Crime in North Carolina"

# 1.b Crime Dataset

In [7]:
data = pd.read_csv('data/crime.csv', index_col=0)
data

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lwfir,lwser,lwmfg,lwfed,lwsta,lwloc,lpctymle,lpctmin,ltaxpc,lmix
1,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.69763,...,5.607452,5.374044,5.434246,6.014619,5.464848,5.444450,-2.433870,3.006608,3.246399,-2.303407
2,1,82,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.87425,...,5.706707,5.444911,5.482013,6.039540,5.536862,5.467174,-2.449038,3.006608,3.213833,-2.272549
3,1,83,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.45144,...,5.736475,5.481292,5.597310,6.084157,5.522900,5.515765,-2.464036,3.006608,3.275311,-2.517281
4,1,84,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.84235,...,5.858180,5.531204,5.640985,6.129421,5.568077,5.577387,-2.478925,3.006608,3.289981,-2.544612
5,1,85,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.14034,...,5.948220,5.564850,5.700042,6.195282,5.639919,5.664972,-2.497306,3.006608,3.337204,-2.372487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,197,83,0.015575,0.226667,0.480392,0.428571,7.77,0.001073,0.869048,18.90585,...,5.540736,5.280478,5.545919,5.865476,5.844326,5.463408,-2.538060,1.697597,2.939471,-2.172773
627,197,84,0.013662,0.204188,1.410260,0.372727,10.11,0.001109,0.872024,22.70475,...,5.569252,5.261022,5.593186,5.846150,5.869890,5.508943,-2.548068,1.697597,3.122574,-2.145931
628,197,85,0.013086,0.180556,0.830769,0.333333,5.96,0.001054,0.875000,24.12361,...,5.604843,5.333961,5.631821,5.880086,5.871498,5.616807,-2.561072,1.697597,3.183191,-2.512306
629,197,86,0.012874,0.112676,2.250000,0.244444,7.68,0.001088,0.880952,24.98198,...,5.743947,5.371892,5.723879,5.931024,5.873919,5.685245,-2.580968,1.697597,3.218155,-2.580217


In [8]:
# Let's create a indicator variable for crime rate (1 if greater than 0.025)

endog = data['crmrte'].apply(lambda x: 1 if x > 0.025 else 0)

In [9]:
# Balanced outcomes

endog.value_counts()

1    363
0    267
Name: crmrte, dtype: int64

In [10]:
# Same exogenous variables as in notebook 2's model

exog = data[['density', 'taxpc', 'wcon', 'pctmin']]

In [11]:
# Set index (entity, time) for PanelOLS's API

data.loc[:, 'outcome'] = endog
panel_data = data.set_index(['county', 'year'])

# 2. Non-linear panel models

## 2.1 Pooled logit

In [12]:
# Pooled logit

logit_mod = sm.Logit(endog=endog, exog=exog)
logit_res = logit_mod.fit()
logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.459943
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 crmrte   No. Observations:                  630
Model:                          Logit   Df Residuals:                      626
Method:                           MLE   Df Model:                            3
Date:                Fri, 12 Mar 2021   Pseudo R-squ.:                  0.3251
Time:                        11:01:24   Log-Likelihood:                -289.76
converged:                       True   LL-Null:                       -429.34
Covariance Type:            nonrobust   LLR p-value:                 3.231e-60
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
density        3.1274      0.307     10.196      0.000       2.526       3.729
taxpc         -0.0036      0.011     -0.333      0.739      -0.025       0.018
wcon          -0.0135      0.002     -6.513      0.000      -0.018      -0.009
pctmin         0.0268      0.005      4.906      0.000       0.016       0.038
==============================================================================
"""

## 2.2 Conditional logit (county fixed effects)

In [13]:
# Conditional logit (country FE)

clogit_country_mod = ConditionalLogit(endog=endog,
                                      exog=exog,
                                      groups=data['county'])
clogit_country_res = clogit_country_mod.fit(method='nm')
clogit_country_res.summary()

/home/topher-lo/repos/PyEC333/ec333_env/lib/python3.8/site-packages/statsmodels/discrete/conditional_models.py:80: UserWarning: Dropped 73 groups and 511 observations for having no within-group variance
  warnings.warn(msg)


<class 'statsmodels.iolib.summary.Summary'>
"""
                  Conditional Logit Model Regression Results                  
==============================================================================
Dep. Variable:                 crmrte   No. Observations:                  119
Model:               ConditionalLogit   No. groups:                         17
Log-Likelihood:               -39.803   Min group size:                      7
Method:                            nm   Max group size:                      7
Date:                Fri, 12 Mar 2021   Mean group size:                   7.0
Time:                        11:01:24                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
density        0.0079     11.742      0.001      0.999     -23.006      23.022
taxpc         -0.0521      0.042     -1.244      0.214      -0.134       0.030
wcon           0.0190      0.012      1.571      0.116      -0.005       0.043
pctmin         0.0092    123.581   7.44e-05      1.000    -242.204     242.223
==============================================================================
"""

### Note:
- Try changing the numerical method to 'bfgs' (which is default method for `statsmodels`'s conditional logit), it will fail to converge

## 2.3 Conditional logit (year fixed effects)

In [14]:
# Conditional logit (year FE)

clogit_year_mod = ConditionalLogit(endog=endog,
                                   exog=exog,
                                   groups=data['year'])
clogit_year_res = clogit_year_mod.fit(method='nm', maxiter=500)
clogit_year_res.summary()

/home/topher-lo/repos/PyEC333/ec333_env/lib/python3.8/site-packages/statsmodels/discrete/conditional_models.py:283: RuntimeWarning: overflow encountered in double_scalars
  v = f(t - 1, k) + f(t - 1, k - 1) * exb[t - 1]


<class 'statsmodels.iolib.summary.Summary'>
"""
                  Conditional Logit Model Regression Results                  
==============================================================================
Dep. Variable:                 crmrte   No. Observations:                  630
Model:               ConditionalLogit   No. groups:                          7
Log-Likelihood:               -233.00   Min group size:                     90
Method:                            nm   Max group size:                     90
Date:                Fri, 12 Mar 2021   Mean group size:                  90.0
Time:                        11:01:32                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
density        4.0975      0.367     11.170      0.000       3.378       4.816
taxpc          0.0286      0.010      2.735      0.006       0.008       0.049
wcon          -0.0007      0.001     -0.652      0.515      -0.003       0.001
pctmin         0.0523      0.007      7.581      0.000       0.039       0.066
==============================================================================
"""

## 2.4 LPM

In [15]:
# Pooled LPM

lpm_mod = sm.OLS(endog=endog,
                 exog=exog)
lpm_res = lpm_mod.fit()
lpm_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 crmrte   R-squared (uncentered):                   0.664
Model:                            OLS   Adj. R-squared (uncentered):              0.662
Method:                 Least Squares   F-statistic:                              308.9
Date:                Fri, 12 Mar 2021   Prob (F-statistic):                   1.54e-146
Time:                        11:01:32   Log-Likelihood:                         -377.00
No. Observations:                 630   AIC:                                      762.0
Df Residuals:                     626   BIC:                                      779.8
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
density        0.1657      0.013     13.212      0.000       0.141       0.190
taxpc          0.0012      0.001      0.978      0.329      -0.001       0.004
wcon           0.0004      0.000      2.826      0.005       0.000       0.001
pctmin         0.0077      0.001      8.129      0.000       0.006       0.010
==============================================================================
Omnibus:                     5541.218   Durbin-Watson:                   0.511
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               58.615
Skew:                          -0.102   Prob(JB):                     1.87e-13
Kurtosis:                       1.520   Cond. No.                         197.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 2.5 LPM (county fixed effects)

In [16]:
# Entity fixed effects

entity_fe_formula = "outcome ~ density + taxpc + wcon + pctmin + EntityEffects"
mod_entity_fe = PanelOLS.from_formula(entity_fe_formula, data=panel_data, drop_absorbed=True)
res_entity_fe = mod_entity_fe.fit()
res_entity_fe.summary

/home/topher-lo/repos/PyEC333/ec333_env/lib/python3.8/site-packages/linearmodels/panel/model.py:1743: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

pctmin

  warnings.warn(


Dep. Variable:,outcome,R-squared:,0.0003
Estimator:,PanelOLS,R-squared (Between):,-0.0274
No. Observations:,630,R-squared (Within):,0.0003
Date:,"Fri, Mar 12 2021",R-squared (Overall):,-0.0260
Time:,11:01:32,Log-likelihood,206.72
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.0601
Entities:,90,P-value,0.9807
Avg Obs:,7.0000,Distribution:,"F(3,537)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,0.0601


## 2.6 LPM (year fixed effects)

In [17]:
# Time period fixed effects

year_fe_formula = "outcome ~ density + taxpc + wcon + pctmin + TimeEffects"
mod_year_fe = PanelOLS.from_formula(year_fe_formula, data=panel_data, drop_absorbed=True)
res_year_fe = mod_year_fe.fit()
res_year_fe.summary

Dep. Variable:,outcome,R-squared:,0.2345
Estimator:,PanelOLS,R-squared (Between):,0.5270
No. Observations:,630,R-squared (Within):,-0.0204
Date:,"Fri, Mar 12 2021",R-squared (Overall):,0.4981
Time:,11:01:32,Log-likelihood,-364.29
Cov. Estimator:,Unadjusted,,
,,F-statistic:,47.403
Entities:,90,P-value,0.0000
Avg Obs:,7.0000,Distribution:,"F(4,619)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,47.403


## 2.7 LPM (county and year fixed effects)

In [18]:
# Entity and time period fixed effects

twoways_fe_formula = "outcome ~ density + taxpc + wcon + pctmin + TimeEffects + EntityEffects"
mod_twoways_fe = PanelOLS.from_formula(twoways_fe_formula, data=panel_data, drop_absorbed=True)
res_twoways_fe = mod_twoways_fe.fit()
res_twoways_fe.summary

Dep. Variable:,outcome,R-squared:,0.0023
Estimator:,PanelOLS,R-squared (Between):,-0.2756
No. Observations:,630,R-squared (Within):,-0.0038
Date:,"Fri, Mar 12 2021",R-squared (Overall):,-0.2613
Time:,11:01:32,Log-likelihood,218.66
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.4037
Entities:,90,P-value,0.7504
Avg Obs:,7.0000,Distribution:,"F(3,531)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,0.4037
